In [1]:
import os
import random
from typing import Tuple

import numpy as np
import torch
from torch import Tensor
from torch import nn
from torch import optim
from torch.backends import cudnn
from torch.nn import functional as F
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import TensorDataset, random_split
from tqdm import tqdm
from torch.autograd import Variable
torch.set_printoptions(precision=4)


def set_seed(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

set_seed(1234)

In [2]:
latent_img_train       = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces_overfit/latent_img_overfit.npy')
latent_img_label_train = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces_overfit/latent_img_label_overfit.npy')


latent_sensor_corrupted_train   = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces_overfit/latent_sensor_corrupted_overfit.npy')
latent_sensor_uncorrupted_train = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces_overfit/latent_sensor_uncorrupted_overfit.npy')
latent_sensor_label_train       = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces_overfit/latent_sensor_label_overfit.npy')


In [4]:
def create_dataset(
    batch_size = 24,
    seed       = 1234,
    img_train  = latent_img_train , 
    img_train_label    = latent_img_label_train,
    sensor_train       = latent_sensor_uncorrupted_train,
    sensor_train_label = latent_sensor_label_train
):
    np.random.seed(seed)

    # x_img    = np.array(img_train) 
    # x_sensor = np.array(sensor_train) 
    # y_img    = np.array(img_train_label)
    # y_sensor = np.array(sensor_train_label)

    x_img    = np.array(img_train)
    x_sensor = np.array(sensor_train)
    y_img    = np.array(img_train_label)
    y_sensor = np.array(sensor_train_label)
    
    tensor_x_img    = torch.Tensor(x_img)
    tensor_x_sensor = torch.Tensor(x_sensor)
    tensor_y_img    = torch.Tensor(y_img)
    tensor_y_sensor = torch.Tensor(y_sensor)
   
    tensor_dataset = TensorDataset(tensor_x_img, tensor_x_sensor, tensor_y_img, tensor_y_sensor)

    train_loader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=True)

    return train_loader

In [5]:
# class Latent_Encoder(nn.Module):
#     def __init__(self, latent_size=256):
#         super(Latent_Encoder, self).__init__()

#         self.encoder = nn.Sequential(
#             nn.Linear(latent_size, 128),
#             nn.ReLU(),
#             nn.Linear(128, 64),
            
#         )
#         self.decoder = nn.Sequential(
#             nn.Linear(64, 128),
#             nn.ReLU(),
#             nn.Linear(128, latent_size),
#             nn.Sigmoid(),
#         )

#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.decoder(x)

#         return x

class Latent_Encoder(nn.Module):
    def __init__(self):
        super(Latent_Encoder, self).__init__()
        self.layer1 = nn.Linear(256, 256)
        self.dropout1 = nn.Dropout(0.25)
        self.layer2 = nn.Linear(256, 256)
    
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.layer2(x)
        
        return x

In [6]:
def get_min_and_max(dataloader, device) -> Tuple[float, float]:
    """
    Finds the min and max of the dataset.
    This is used for Normalization of the dataset.

    :param dataloader: dataloader to calculate it for
    :param device: device to run computations on
    :return: tuple of mean and std
    """
    min_val_img, max_val_img = torch.Tensor([999]).to(device), torch.Tensor([-999]).to(device)
    min_val_sensor, max_val_sensor = torch.Tensor([999]).to(device), torch.Tensor([-999]).to(device)
    
    for data in tqdm(dataloader):
        x_img    = data[0].to(device)
        x_sensor = data[1].to(device)

        min_val_img = torch.min(min_val_img, torch.min(x_img))
        max_val_img = torch.max(max_val_img, torch.max(x_img))

        min_val_sensor = torch.min(min_val_sensor, torch.min(x_sensor))
        max_val_sensor = torch.max(max_val_sensor, torch.max(x_sensor))

    return min_val_img.item(), max_val_img.item(), min_val_sensor.item(), max_val_sensor.item()

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Latent_Encoder().cuda()
train_loader = create_dataset()
# min_val_img, max_val_img, min_val_sensor, max_val_sensor = get_min_and_max(train_loader, device)

In [8]:
optimizer = optim.Adam(model.parameters(), lr = 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)

In [10]:
# for data in tqdm(train_loader):
#     x_img, x_sensor, y_img, y_sensor = data
#     break

  0%|          | 0/1 [00:00<?, ?it/s]


In [9]:
for epoch in range(250):
  train_loss = 0
  for data in tqdm(train_loader):
  
      x_img, x_sensor, y_img, y_sensor = data
      
      # x_img = x_img.to(device)
      # x_img = (x_img - min_val_img) / (max_val_img - min_val_img)

      # x_sensor = x_sensor.to(device)
      # x_sensor = (x_sensor - min_val_sensor) / (max_val_sensor - min_val_sensor)

      x_img = x_img.to(device)
      x_sensor = x_sensor.to(device)

      optimizer.zero_grad()
      y = model(x_sensor)
      loss = F.mse_loss(x_img, y)
 
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
      
  # if(epoch%100 == 0):
      scheduler.step(train_loss/len(train_loader))
      print("Epoch: {} || Train Loss: {}".format(epoch + 1, train_loss / len(train_loader)))

100%|██████████| 1/1 [00:01<00:00,  1.60s/it]


Epoch: 1 || Train Loss: 1.000784158706665


100%|██████████| 1/1 [00:00<00:00, 180.08it/s]


Epoch: 2 || Train Loss: 0.9832314252853394


100%|██████████| 1/1 [00:00<00:00, 88.72it/s]


...

Epoch: 248 || Train Loss: 0.06986211985349655



100%|██████████| 1/1 [00:00<00:00, 232.35it/s]

Epoch: 249 || Train Loss: 0.05906256288290024



100%|██████████| 1/1 [00:00<00:00, 195.33it/s]

Epoch: 250 || Train Loss: 0.07424245774745941


In [10]:
x = train_loader.dataset[0][0].cuda()
y = train_loader.dataset[0][1].cuda()

In [11]:
model.eval()

Latent_Encoder(
  (layer1): Linear(in_features=256, out_features=256, bias=True)
  (dropout1): Dropout(p=0.25, inplace=False)
  (layer2): Linear(in_features=256, out_features=256, bias=True)
)

In [12]:
yy = model(y)

In [13]:
y[0,:10], yy[0,:10], x[0,:10]

(tensor([-0.3057,  0.1903,  0.1098, -0.0039, -0.0821, -0.0503, -0.0778, -0.1800,
         -0.2827, -0.5074], device='cuda:0'),
 tensor([ 0.8263, -0.1333,  1.7725,  0.7543, -1.6729, -0.5638, -0.4758, -0.3843,
         -0.5492, -0.8176], device='cuda:0', grad_fn=<SliceBackward>),
 tensor([ 0.8002, -0.2443,  2.0559,  0.7736, -1.9683, -0.6645, -0.5672, -0.4281,
         -0.7702, -0.9821], device='cuda:0'))

In [16]:
torch.save(model.state_dict(),'/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/models/ltl_big.pt')